### Calculating Null Distribution

Code modified from @adeboyeML

Null distribution - is generated by getting the median correlation score of randomly combined replicates that do not come from the same compounds.

### The goal here: 

-- is to compute the **p-value** for each compound per dose by evaluating the probability of random combinations of replicates (from different compounds) having greater median correlation score than replicates that come from the same compound.

- In our case, we generated 1000 median correlation scores from randomly combined replicates as the **null distribution** for each no_of_replicates/replicate class - we have 1000 medians scores from randomly combined replicates of different compounds.

**no_of_replicate** is the number of replicates in a specific compound and **no_of_replicate class** is a specific group of compounds that have the same amount of replicates e.g all compounds with 3 replicates in them are in the same no_of_replicates class.

In [1]:
import os
import pathlib
import requests
import pickle
import pandas as pd
import numpy as np
import re
from os import walk
from collections import Counter
import random
import shutil
from statistics import median
import cmapPy.pandasGEXpress.parse_gct as pg
from cmapPy.pandasGEXpress.parse import parse

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

In [2]:
np.random.seed(42)

In [3]:
# Load common compounds
common_file = pathlib.Path(
    "..", "..", "..", "6.paper_figures", "data", "significant_compounds_by_threshold_both_assays.tsv.gz"
)
common_df = pd.read_csv(common_file, sep="\t")

common_compounds = common_df.compound.unique()
print(len(common_compounds))

1327


### - Load in Level 4 Datasets generated from `calculate_median_scores_notebook`

In [4]:
# Run either using "W" or ""
# Representing "whitened" (aka "spherized") L1000 data or not
l1000_file_indicator = ""

In [5]:
L1000_level4_path = "L1000_lvl4_cpd_replicate_datasets"

In [6]:
df_level4 = pd.read_csv(
    os.path.join(L1000_level4_path, f'L1000_level4{l1000_file_indicator}_cpd_replicates.csv.gz'), 
    compression='gzip',
    low_memory = False
)
print(df_level4.shape)
df_level4.head()

(27837, 988)


,replicate_id,200814_at,222103_at,201453_x_at,204131_s_at,200059_s_at,205067_at,213702_x_at,214435_x_at,201334_s_at,...,205379_at,sig_id,pert_id,pert_idose,det_plate,det_well,dose,Metadata_broad_sample,pert_iname,moa
0,REP.A001_A549_24H_X1_B27:A03,0.3547,-0.4940,-0.1721,-0.0339,-0.4355,1.8263,-0.1316,0.0853,-0.4660,...,0.1046,REP.A001_A549_24H:A03,DMSO,-666,REP.A001_A549_24H_X1_B27|REP.A001_A549_24H_X2_...,A03,0,DMSO,DMSO,Control vehicle
1,REP.A001_A549_24H_X2_B27:A03,-0.0447,-1.6390,-0.5276,-0.5092,-0.5733,0.2445,0.6159,-1.1273,2.8250,...,-1.5836,REP.A001_A549_24H:A03,DMSO,-666,REP.A001_A549_24H_X1_B27|REP.A001_A549_24H_X2_...,A03,0,DMSO,DMSO,Control vehicle
2,REP.A001_A549_24H_X3_B27:A03,-0.9583,0.2657,-0.8204,-0.5851,-2.0808,0.0739,-0.0497,-0.3012,-0.9787,...,-0.1416,REP.A001_A549_24H:A03,DMSO,-666,REP.A001_A549_24H_X1_B27|REP.A001_A549_24H_X2_...,A03,0,DMSO,DMSO,Control vehicle
3,REP.A001_A549_24H_X1_B27:A04,-0.2130,0.4931,-0.8768,-0.6968,-1.7018,-0.3779,-0.6745,-1.9799,-1.1429,...,-1.8243,REP.A001_A549_24H:A04,DMSO,-666,REP.A001_A549_24H_X1_B27|REP.A001_A549_24H_X2_...,A04,0,DMSO,DMSO,Control vehicle
4,REP.A001_A549_24H_X2_B27:A04,0.7499,-1.2819,0.4981,1.7090,1.6765,-1.2690,1.7974,-1.2213,3.4625,...,0.3157,REP.A001_A549_24H:A04,DMSO,-666,REP.A001_A549_24H_X1_B27|REP.A001_A549_24H_X2_...,A04,0,DMSO,DMSO,Control vehicle


In [7]:
# Load compound median scores
if l1000_file_indicator == "W":
    file = os.path.join(L1000_level4_path, 'cpd_replicate_median_scores_w.csv')
else:
    file = os.path.join(L1000_level4_path, 'cpd_replicate_median_scores.csv')

df_cpd_med_scores = pd.read_csv(file)
df_cpd_med_scores = df_cpd_med_scores.set_index('cpd').rename_axis(None, axis=0).copy()

# Subset to common compound measurements
df_cpd_med_scores = df_cpd_med_scores.loc[df_cpd_med_scores.index.isin(common_compounds), :]

print(df_cpd_med_scores.shape)
df_cpd_med_scores.head()

(1258, 7)


,dose_1,dose_2,dose_3,dose_4,dose_5,dose_6,no_of_replicates
17-hydroxyprogesterone-caproate,0.073370,0.064999,0.022398,0.149151,0.031656,0.226888,3
2-iminobiotin,0.085434,0.196718,0.012828,0.005784,0.046477,0.128650,2
3-amino-benzamide,0.011228,-0.000293,0.052259,0.013569,-0.013665,0.021126,3
3-deazaadenosine,0.068822,0.001870,0.045240,0.204147,0.061483,0.049968,2
abacavir,0.055853,0.047647,0.170106,0.071078,0.033011,0.080949,3


In [8]:
def get_cpds_replicates(df, df_lvl4):
    """
    This function returns all replicates id/names found in each compound
    """
        
    replicates_in_all = []
    cpds_replicates = {}
    for cpd in df.index.unique():
        replicate_names = df_lvl4[df_lvl4['pert_iname'] == cpd]['replicate_id'].values.tolist()
        replicates_in_all += replicate_names
        cpds_replicates[cpd] = replicate_names
        
    return replicates_in_all, cpds_replicates

In [9]:
replicates_in_all, cpds_replicates = get_cpds_replicates(df_cpd_med_scores, df_level4)

In [10]:
def get_replicates_classes(df, df_lvl4, cpds_replicates):
    
    """
    This function gets all replicates ids for each distinct 
    no_of_replicates (i.e. number of replicates per cpd) class
    
    Returns replicate_class_dict dictionary, with no_of_replicate classes as the keys, 
    and all the replicate_ids for each no_of_replicate class as the values
    """
    
    df['replicate_id'] = list(cpds_replicates.values())
    replicate_class_dict = {}
    for size in df['no_of_replicates'].unique():
        rep_lists = []
        for idx in range(df[df['no_of_replicates'] == size].shape[0]):
            rep_ids = df[df['no_of_replicates'] == size]['replicate_id'].values.tolist()[idx]
            rep_lists += rep_ids
            replicate_class_dict[size] = rep_lists
                
    return replicate_class_dict

In [11]:
cpd_replicate_class_dict = get_replicates_classes(df_cpd_med_scores, df_level4, cpds_replicates)

In [12]:
cpd_replicate_class_dict.keys()

dict_keys([3, 2, 4, 6])

In [13]:
def check_similar_replicates(replicates, cpd_dict):
    """This function checks if two replicates are of the same compounds"""
    
    for x in range(len(replicates)):
        for y in range(x+1, len(replicates)):
            for kys in cpd_dict:
                if all(i in cpd_dict[kys] for i in [replicates[x], replicates[y]]):
                    return True
    return False

In [14]:
def get_random_replicates(all_replicates, no_of_replicates, replicates_ids, cpd_replicate_dict):
    """
    This function return a list of random replicates that are not of the same compounds
    or found in the current cpd's size list
    """
    while (True):
        random_replicates = random.sample(all_replicates, no_of_replicates)
        if not (any(rep in replicates_ids for rep in random_replicates) & 
                (check_similar_replicates(random_replicates, cpd_replicate_dict))):
            break
    return random_replicates

In [15]:
def get_null_distribution_replicates(
    cpd_replicate_class_dict,
    replicates_lists,
    cpd_replicate_dict,
    rand_num = 1000
):
    
    """
    This function returns a null distribution dictionary, with no_of_replicates(replicate class) 
    as the keys and 1000 lists of randomly selected replicate combinations as the values
    for each no_of_replicates class
    """
    random.seed(1903)
    null_distribution_reps = {}
    for replicate_class in cpd_replicate_class_dict:
        replicates_ids = cpd_replicate_class_dict[replicate_class]
        replicate_list = []
        for idx in range(rand_num):
            start_again = True
            while (start_again):
                rand_cpds = get_random_replicates(
                    replicates_lists,
                    replicate_class,
                    replicates_ids,
                    cpd_replicate_dict
                )
                if rand_cpds not in replicate_list:
                    start_again = False
            replicate_list.append(rand_cpds)
        null_distribution_reps[replicate_class] = replicate_list
    
    return null_distribution_reps

In [16]:
len(cpds_replicates.keys())

1258

In [17]:
null_distribution_replicates = get_null_distribution_replicates(
    cpd_replicate_class_dict,
    replicates_in_all,
    cpds_replicates
)

In [18]:
def save_to_pickle(null_distribution, path, file_name):
    """This function saves the null distribution replicates ids into a pickle file"""
    
    if not os.path.exists(path):
        os.mkdir(path)
        
    with open(os.path.join(path, file_name), 'wb') as handle:
        pickle.dump(null_distribution, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
# Save results
if l1000_file_indicator == "W":
    file = 'null_distribution_w_dose_independent.pickle'
else:
    file = 'null_distribution_dose_independent.pickle'

save_to_pickle(null_distribution_replicates, L1000_level4_path, file)

In [20]:
##load the null_distribution_moa from pickle
with open(os.path.join(L1000_level4_path, file), 'rb') as handle:
    null_distribution_replicates = pickle.load(handle)

In [21]:
def assert_null_distribution(null_distribution_reps):
    """
    This function assert that each of the list in the 1000 lists of random replicate 
    combination for each no_of_replicate class are distinct with no duplicates
    """
    duplicates_reps = {}
    for keys in null_distribution_reps:
        null_dist = null_distribution_reps[keys]
        for reps in null_dist:
            dup_reps = []
            new_list = list(filter(lambda x: x != reps, null_dist))
            if (len(new_list) != len(null_dist) - 1):
                dup_reps.append(reps)
        if dup_reps:
            if keys not in duplicates_reps:
                duplicates_reps[keys] = [dup_reps]
            else:
                duplicates_reps[keys] += [dup_reps]
    return duplicates_reps

In [22]:
duplicate_replicates = assert_null_distribution(null_distribution_replicates)

In [23]:
duplicate_replicates ##no duplicates

{}

In [24]:
def calc_null_dist_median_scores(df, replicate_lists):
    """
    This function calculate the median of the correlation 
    values for each list in the 1000 lists of random replicate 
    combination for each no_of_replicate class per dose
    """
    df = df.set_index('replicate_id').rename_axis(None, axis=0)
    df.drop(['Metadata_broad_sample', 'pert_id', 'dose', 'pert_idose', 
             'pert_iname', 'moa', 'sig_id', 'det_plate', 'det_well'], axis = 1, inplace = True)
    median_corr_list = []
    for rep_list in replicate_lists:
        df_reps = df.loc[rep_list].copy()
        reps_corr = df_reps.astype('float64').T.corr(method = 'spearman').values
        median_corr_val = median(list(reps_corr[np.triu_indices(len(reps_corr), k = 1)]))
        median_corr_list.append(median_corr_val)
    return median_corr_list

In [25]:
def get_null_dist_median_scores(null_distribution_cpds, df):
    """ 
    This function calculate the median correlation scores for all 
    1000 lists of randomly combined compounds for each no_of_replicate class 
    """
    null_distribution_medians = {}
    for key in null_distribution_cpds:
        replicate_median_scores = calc_null_dist_median_scores(df, null_distribution_cpds[key])
        null_distribution_medians[key] = replicate_median_scores
    return null_distribution_medians

In [26]:
null_distribution_medians = get_null_dist_median_scores(null_distribution_replicates, df_level4)

In [27]:
def compute_dose_median_scores(null_dist_medians, dose_list):
    """
    This function align median scores per dose, and return a dictionary, 
    with keys as dose numbers and values as all median null distribution/non-replicate correlation 
    scores for each dose
    """
    median_scores_per_dose = {}
    for dose in dose_list:
        median_list = []
        for keys in null_distribution_medians:
            dose_median_list = null_distribution_medians[keys]
            median_list += dose_median_list
        median_scores_per_dose[dose] = median_list
    return median_scores_per_dose

In [28]:
# Extract dose_list only to index in median_score data
dose_list = list(set(df_level4['dose'].unique().tolist()))[1:7]

dose_null_medians = compute_dose_median_scores(null_distribution_medians, dose_list)

In [29]:
#save the null_distribution_medians_per_dose to pickle
if l1000_file_indicator == "W":
    file = 'null_dist_medians_per_dose_w_dose_independent.pickle'
else:
    file = 'null_dist_medians_per_dose_dose_independent.pickle'

    save_to_pickle(dose_null_medians, L1000_level4_path, file)

**A P value can be computed nonparametrically by evaluating the probability of random replicates of different compounds having median similarity value greater than replicates of the same compounds.**

In [30]:
def get_p_value(median_scores_list, df, dose_name, cpd_name):
    """
    This function calculate the p-value from the 
    null_distribution median scores for each compound
    """
    actual_med = df.loc[cpd_name, dose_name]
    p_value = np.sum(median_scores_list >= actual_med) / len(median_scores_list)
    return p_value

In [31]:
def get_moa_p_vals(null_dist_median, dose_list, df_med_values):
    """
    This function returns a dict, with compounds as the keys and the compound's 
    p-values as the values
    """
    null_p_vals = {}
    for key in null_dist_median:
        df_replicate_class = df_med_values[df_med_values['no_of_replicates'] == key]
        for cpd in df_replicate_class.index:
            dose_p_values = []
            for num in dose_list:
                dose_name = 'dose_' + str(num)
                cpd_p_value = get_p_value(null_dist_median[key], df_replicate_class, dose_name, cpd)
                dose_p_values.append(cpd_p_value)
            null_p_vals[cpd] = dose_p_values
    sorted_null_p_vals = {key:value for key, value in sorted(null_p_vals.items(), key=lambda item: item[0])}
    return sorted_null_p_vals

In [32]:
null_p_vals = get_moa_p_vals(null_distribution_medians, dose_list, df_cpd_med_scores)

In [33]:
df_null_p_vals = pd.DataFrame.from_dict(null_p_vals, orient='index', columns = ['dose_' + str(x) for x in dose_list])

In [34]:
df_null_p_vals['no_of_replicates'] = df_cpd_med_scores['no_of_replicates']

In [35]:
df_null_p_vals.head(10)

,dose_1,dose_2,dose_3,dose_4,dose_5,dose_6,no_of_replicates
17-hydroxyprogesterone-caproate,0.080,0.104,0.287,0.002,0.228,0.001,3
2-iminobiotin,0.147,0.028,0.414,0.454,0.266,0.076,2
3-amino-benzamide,0.350,0.443,0.133,0.335,0.557,0.295,3
3-deazaadenosine,0.198,0.473,0.269,0.025,0.216,0.252,2
abacavir,0.124,0.154,0.002,0.086,0.224,0.068,3
abiraterone-acetate,0.386,0.015,0.375,0.033,0.319,0.125,3
abt-202,0.660,0.107,0.550,0.172,0.814,0.120,3
abt-239,0.044,0.067,0.048,0.266,0.273,0.001,3
abt-724,0.191,0.025,0.111,0.262,0.230,0.001,3
acarbose,0.395,0.256,0.067,0.002,0.246,0.064,3


In [36]:
def save_to_csv(df, path, file_name):
    """saves dataframes to csv"""
    
    if not os.path.exists(path):
        os.mkdir(path)
    
    df.to_csv(os.path.join(path, file_name), index = False)

In [37]:
# Save the null_distribution_medians_per_dose to pickle
if l1000_file_indicator == "W":
    file = 'cpd_replicate_p_values_w_dose_independent.csv'
else:
    file = 'cpd_replicate_p_values_dose_independent.csv'

save_to_csv(df_null_p_vals.reset_index().rename({'index':'cpd'}, axis = 1), L1000_level4_path, file)

In [38]:
if l1000_file_indicator == "W":
    cpd_summary_file = pathlib.Path(L1000_level4_path, 'cpd_replicate_p_values_melted_w_dose_independent.csv')
    input_data = "spherized"
else:
    cpd_summary_file = pathlib.Path(L1000_level4_path, 'cpd_replicate_p_values_melted_dose_independent.csv')
    input_data = "non_spherized"


dose_recode_info = {
    'dose_1': '0.04 uM', 'dose_2':'0.12 uM', 'dose_3':'0.37 uM',
    'dose_4': '1.11 uM', 'dose_5':'3.33 uM', 'dose_6':'10 uM'
}

# Melt the p values
cpd_score_summary_pval_df = (
    df_null_p_vals
    .reset_index()
    .rename(columns={"index": "compound"})
    .melt(
        id_vars=["compound", "no_of_replicates"],
        value_vars=["dose_1", "dose_2", "dose_3", "dose_4", "dose_5", "dose_6"],
        var_name="dose",
        value_name="p_value"
    )
)

cpd_score_summary_pval_df.dose = cpd_score_summary_pval_df.dose.replace(dose_recode_info)

# Melt the median matching scores
cpd_score_summary_df = (
    df_cpd_med_scores
    .reset_index()
    .rename(columns={"index": "compound"})
    .melt(
        id_vars=["compound", "no_of_replicates"],
        value_vars=["dose_1", "dose_2", "dose_3", "dose_4", "dose_5", "dose_6"],
        var_name="dose",
        value_name="matching_score"
    )

)

cpd_score_summary_df.dose = cpd_score_summary_df.dose.replace(dose_recode_info)

summary_df = (
    cpd_score_summary_pval_df
    .merge(cpd_score_summary_df, on=["compound", "no_of_replicates", "dose"], how="inner")
    .assign(
        assay="L1000",
        normalization=input_data,
        category="all_data"
    )
)

summary_df.to_csv(cpd_summary_file, sep="\t", index=False)

print(summary_df.shape)
summary_df.head()

(7548, 8)


,compound,no_of_replicates,dose,p_value,matching_score,assay,normalization,category
0,17-hydroxyprogesterone-caproate,3,0.04 uM,0.080,0.073370,L1000,non_spherized,all_data
1,2-iminobiotin,2,0.04 uM,0.147,0.085434,L1000,non_spherized,all_data
2,3-amino-benzamide,3,0.04 uM,0.350,0.011228,L1000,non_spherized,all_data
3,3-deazaadenosine,2,0.04 uM,0.198,0.068822,L1000,non_spherized,all_data
4,abacavir,3,0.04 uM,0.124,0.055853,L1000,non_spherized,all_data
